# Generador de Folletos Empresariales con LLMs

Este notebook demuestra cómo utilizar un modelo de lenguaje (GPT-4o mini de OpenAI) junto con web scraping en Python para construir automáticamente un **folleto empresarial** a partir del contenido de un sitio web.

##  Caso de estudio: [IMATEC](https://imatec.com.ar)

### ¿Qué hace este proyecto?

1. Realiza **web scraping** con `BeautifulSoup` para extraer el texto y los enlaces relevantes de la página.
2. Usa la API de OpenAI (`gpt-4o-mini`) para:
   - Clasificar los enlaces relevantes para un folleto (precios, contacto, características, etc).
   - Generar el contenido del folleto en **formato Markdown**.
3. Muestra el resultado de forma interactiva (con posibilidad de streaming).

### Posibles aplicaciones
- Automatización de folletos corporativos
- Presentaciones de ventas basadas en contenido web
- Resúmenes ejecutivos de startups a partir de sus landing pages

In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Inicialización y constantes

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("Puede haber un problema con tu clave API")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

La clave de API parece buena


In [4]:
# La clase para representar una Página Web

class Website:
    """
    Una clase para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [5]:
imatec = Website("https://imatec.com.ar/")
print(imatec.get_contents())
imatec.links

Título de la Web:
Imatec - Plataforma Cloud PACS para Estudios Médicos | Visor DICOM Multidisciplinario
Contenido de la Web:
ES
EN
Contactar
Plataforma Cloud para Gestión de Estudios Médicos
Sube, visualiza y comparte estudios DICOM desde cualquier dispositivo. Visor multidisciplinario sin instalaciones para clínicas y centros de diagnóstico.
Solicitar demo gratuita
¿Qué es Imatec?
Imatec es una plataforma cloud PACS que permite a clínicas, consultorios y centros de diagnóstico gestionar y visualizar estudios médicos de forma segura y eficiente, sin necesidad de instalaciones complejas.
¿Cómo funciona?
1
📤
Subir
Carga estudios DICOM desde cualquier dispositivo
2
👁️
Visualizar
Accede al visor avanzado desde tu navegador
3
👥
Acceder
Diferentes roles para médicos, pacientes y personal
4
🔗
Compartir
Enlaces seguros para compartir estudios
Paneles por tipo de usuario
👤
Paciente
Acceso directo a sus estudios con interfaz simplificada
👨‍⚕️
Médico informante
Herramientas completas para diagnós

['?lang=es',
 '?lang=en',
 'mailto:info@medytec.com',
 'mailto:info@medytec.com?subject=Solicitud%20de%20Demo%20Gratuita',
 'mailto:info@medytec.com',
 'mailto:info@medytec.com',
 'mailto:info@medytec.com',
 '#features',
 '#pricing',
 '#faq',
 '#demo',
 'mailto:info@medytec.com',
 'https://www.medytec.com',
 '#',
 '#',
 '#',
 '/privacy',
 '/terms',
 '/cookies']

In [6]:
link_system_prompt = "Se te proporciona una lista de enlaces que se encuentran en una página web. \
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, \
como enlaces a un contacto, una email de la empresa, tipos de planes y precios, FAQ.\n"
link_system_prompt += "Debes responder en JSON como en este ejemplo:"
link_system_prompt += """
{
    "links": [
        {"type": "Caracteristicas", "url": "https://imatec.com.ar/#features"},
        {"type": "Precios": "url": "https://imatec.com.ar/#pricing"}
    ]
}
"""

In [7]:
print(link_system_prompt)

Se te proporciona una lista de enlaces que se encuentran en una página web. Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, como enlaces a un contacto, una email de la empresa, tipos de planes y precios, FAQ.
Debes responder en JSON como en este ejemplo:
{
    "links": [
        {"type": "Caracteristicas", "url": "https://imatec.com.ar/#features"},
        {"type": "Precios": "url": "https://imatec.com.ar/#pricing"}
    ]
}



In [25]:
def get_links_user_prompt(website):
    user_prompt = f"Aquí hay una lista de enlaces de la página web {website.url} - "
    user_prompt += "Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. \
No incluyas Términos y Condiciones, Solo algunos email, enlaces de correo electrónico.\n"
    user_prompt += "Links (puede que algunos sean links relativos):\n"
    user_prompt += "El link, url, principal es https://imatec.com.ar/ :\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [26]:
print(get_links_user_prompt(imatec))

Aquí hay una lista de enlaces de la página web https://imatec.com.ar/ - Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. No incluyas Términos y Condiciones, Solo algunos email, enlaces de correo electrónico.
Links (puede que algunos sean links relativos):
El link, url, principal es https://imatec.com.ar/ :
?lang=es
?lang=en
mailto:info@medytec.com
mailto:info@medytec.com?subject=Solicitud%20de%20Demo%20Gratuita
mailto:info@medytec.com
mailto:info@medytec.com
mailto:info@medytec.com
#features
#pricing
#faq
#demo
mailto:info@medytec.com
https://www.medytec.com
#
#
#
/privacy
/terms
/cookies


In [27]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"} #La api da la respuesta en formato json. Mencionar previamente en el prompt que se requiere una reespuesta tipo json.
    )
    result = response.choices[0].message.content # Obtiene el contenido de texto de la primera respuesta generada por el modelo de OpenAI.
    return json.loads(result)

In [28]:
imatec = Website("https://imatec.com.ar/")
imatec.links

['?lang=es',
 '?lang=en',
 'mailto:info@medytec.com',
 'mailto:info@medytec.com?subject=Solicitud%20de%20Demo%20Gratuita',
 'mailto:info@medytec.com',
 'mailto:info@medytec.com',
 'mailto:info@medytec.com',
 '#features',
 '#pricing',
 '#faq',
 '#demo',
 'mailto:info@medytec.com',
 'https://www.medytec.com',
 '#',
 '#',
 '#',
 '/privacy',
 '/terms',
 '/cookies']

In [29]:
get_links("https://imatec.com.ar/")

{'links': [{'type': 'Email de contacto', 'url': 'mailto:info@medytec.com'},
  {'type': 'Email para solicitud de demo',
   'url': 'mailto:info@medytec.com?subject=Solicitud%20de%20Demo%20Gratuita'},
  {'type': 'Caracteristicas', 'url': 'https://imatec.com.ar/#features'},
  {'type': 'Precios', 'url': 'https://imatec.com.ar/#pricing'},
  {'type': 'FAQ', 'url': 'https://imatec.com.ar/#faq'},
  {'type': 'Demo', 'url': 'https://imatec.com.ar/#demo'}]}

In [32]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Links encontrados:", links)
    for link in links["links"]:
        if link["url"].startswith("http"):  # Filtra solo URLs válidas
            result += f"\n\n{link['type']}\n"
            result += Website(link["url"]).get_contents()
        else:
            result += f"\n\n{link['type']} (no se puede acceder a este enlace: {link['url']})\n"
    return result

In [33]:
print(get_all_details("https://imatec.com.ar"))

Links encontrados: {'links': [{'type': 'Email de contacto', 'url': 'mailto:info@medytec.com'}, {'type': 'Solicitud de Demo Gratuita', 'url': 'mailto:info@medytec.com?subject=Solicitud%20de%20Demo%20Gratuita'}, {'type': 'Caracteristicas', 'url': 'https://imatec.com.ar/#features'}, {'type': 'Precios', 'url': 'https://imatec.com.ar/#pricing'}, {'type': 'FAQ', 'url': 'https://imatec.com.ar/#faq'}, {'type': 'Demo', 'url': 'https://imatec.com.ar/#demo'}]}
Landing page:
Título de la Web:
Imatec - Plataforma Cloud PACS para Estudios Médicos | Visor DICOM Multidisciplinario
Contenido de la Web:
ES
EN
Contactar
Plataforma Cloud para Gestión de Estudios Médicos
Sube, visualiza y comparte estudios DICOM desde cualquier dispositivo. Visor multidisciplinario sin instalaciones para clínicas y centros de diagnóstico.
Solicitar demo gratuita
¿Qué es Imatec?
Imatec es una plataforma cloud PACS que permite a clínicas, consultorios y centros de diagnóstico gestionar y visualizar estudios médicos de forma 

In [34]:
system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa\
y crea un folleto breve sobre la empresa para posibles clientes e inversores. Responde en formato Markdown.\
Incluye detalles la empresa desarrolladora, el sistema, los clientes, los planes y precios, caracteristicas, como puede ayudarte si sos medico y tenes una clinica, sanatorio, hospital, etc, que valor agregado te puede dar y todo relevante para vender el sistema"

In [35]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"Estás mirando una empresa llamada: {company_name}\n"
    user_prompt += f"Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\n"
    user_prompt += get_all_details(url)
    return user_prompt

In [36]:
get_brochure_user_prompt("Imatec", "https://imatec.com.ar/")

Links encontrados: {'links': [{'type': 'Email de contacto', 'url': 'mailto:info@medytec.com'}, {'type': 'Solicitud de Demo Gratuita', 'url': 'mailto:info@medytec.com?subject=Solicitud%20de%20Demo%20Gratuita'}, {'type': 'Características', 'url': 'https://imatec.com.ar/#features'}, {'type': 'Precios', 'url': 'https://imatec.com.ar/#pricing'}, {'type': 'FAQ', 'url': 'https://imatec.com.ar/#faq'}]}


'Estás mirando una empresa llamada: Imatec\nAquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\nLanding page:\nTítulo de la Web:\nImatec - Plataforma Cloud PACS para Estudios Médicos | Visor DICOM Multidisciplinario\nContenido de la Web:\nES\nEN\nContactar\nPlataforma Cloud para Gestión de Estudios Médicos\nSube, visualiza y comparte estudios DICOM desde cualquier dispositivo. Visor multidisciplinario sin instalaciones para clínicas y centros de diagnóstico.\nSolicitar demo gratuita\n¿Qué es Imatec?\nImatec es una plataforma cloud PACS que permite a clínicas, consultorios y centros de diagnóstico gestionar y visualizar estudios médicos de forma segura y eficiente, sin necesidad de instalaciones complejas.\n¿Cómo funciona?\n1\n📤\nSubir\nCarga estudios DICOM desde cualquier dispositivo\n2\n👁️\nVisualizar\nAccede al visor avanzado desde tu navegador\n3\n👥\nAcceder\nDiferentes roles p

In [37]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [38]:
create_brochure("Imatec", "https://imatec.com.ar")

Links encontrados: {'links': [{'type': 'Email de contacto', 'url': 'mailto:info@medytec.com'}, {'type': 'Solicitud de Demo Gratuita', 'url': 'mailto:info@medytec.com?subject=Solicitud%20de%20Demo%20Gratuita'}, {'type': 'Características', 'url': 'https://imatec.com.ar/#features'}, {'type': 'Precios', 'url': 'https://imatec.com.ar/#pricing'}, {'type': 'FAQ', 'url': 'https://imatec.com.ar/#faq'}, {'type': 'Demo', 'url': 'https://imatec.com.ar/#demo'}]}


# Imatec - Plataforma Cloud PACS para Estudios Médicos

## ¿Qué es Imatec?
Imatec es una innovadora plataforma cloud PACS que permite a clínicas, consultorios y centros de diagnóstico gestionar y visualizar estudios médicos de forma segura y eficiente, sin la necesidad de instalaciones complejas.

---

## ¿Cómo Funciona?
1. **📤 Subir**: Carga estudios DICOM desde cualquier dispositivo.
2. **👁️ Visualizar**: Accede al visor avanzado directamente desde tu navegador.
3. **👥 Acceder**: Diferentes roles para médicos, pacientes y personal administrativo.
4. **🔗 Compartir**: Enlaces seguros para compartir estudios clínicos.

---

## Paneles por Tipo de Usuario
- **Paciente**: Acceso directo a sus estudios con una interfaz simplificada.
- **Médico Informante**: Herramientas completas para diagnóstico y elaboración de informes.
- **Médico Derivante**: Visualización de estudios solicitados y resultados.
- **Secretaria/Admin**: Gestión completa de usuarios, estudios y configuración.

---

## Características Destacadas
- **📱 Accesible Desde Cualquier Dispositivo**: Funciona en desktop, tablets y móviles.
- **🌐 Sin Instalaciones**: Todo opera desde el navegador, sin software adicional.
- **👥 Roles de Usuario**: Acceso personalizado basado en el tipo de usuario.
- **🔒 Enlaces Seguros**: Comparte estudios de forma segura y confiable.
- **✨ Mejor Experiencia del Paciente**: Interfaz intuitiva y fácil de usar.

---

## Planes y Precios
### Elige el plan que mejor se adapte a tu centro médico

- **Básico - $99/mes**
  - Máquina virtual en la nube
  - 4GB RAM
  - 2 vCPU Intel
  - Visor DICOM
  - 100GB de espacio
  - 5 usuarios
  - Soporte por email

- **Profesional - $199/mes (Más Popular)**
  - Lo que incluye el plan básico
  - 500GB de espacio
  - 15 usuarios
  - Soporte prioritario
  - Herramientas avanzadas

- **Empresarial - Personalizado**
  - Soporte 24/7
  - Integración personalizada

---

## FAQs
- **¿Qué tipo de soporte ofrecen?**  
  Ofrecemos soporte técnico 24/7 vía email y chat en vivo para resolver cualquier consulta.

- **¿Qué tan segura es la plataforma?**  
  Utilizamos encriptación de extremo a extremo y cumplimos con estándares internacionales de seguridad médica.

- **¿En qué dispositivos funciona?**  
  Funciona en cualquier dispositivo con navegador web: PC, Mac, tablets iOS/Android y smartphones.

- **¿Se integra con otros sistemas?**  
  Sí, ofrecemos integraciones con sistemas de gestión hospitalaria (HIS) y otros sistemas médicos.

---

## ¿Cómo Puede Ayudarte Imatec?
- **Eficiencia en el Trabajo**: Reduce el tiempo de espera para la visualización de estudios, mejorando la productividad en tu clínica o sanatorio.
- **Mejora en la Atención al Paciente**: Acceso inmediato a los estudios genera confianza y satisfacción en los pacientes.
- **Seguridad y Cumplimiento**: Cumple con las normativas de seguridad médica, asegurando la privacidad de los datos del paciente.

---

## Contacto 
Para más información, solicita una **demo gratuita** o contáctanos:
- **Email**: [info@medytec.com](mailto:info@medytec.com)
- **Ubicación**: Buenos Aires, Argentina
- **Web**: [www.medytec.com](http://www.medytec.com)

---

**Medytec**, la empresa detrás de Imatec, desarrolla tecnología médica simple y confiable desde Argentina. ¡Únete a la revolución digital en la salud!

In [43]:
# Por último, una pequeña mejora. Con un pequeño ajuste, podemos cambiar esto para que los resultados se transmitan desde OpenAI, con la animación de máquina de escribir habitual

def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [46]:
stream_brochure("Imatec", "https://imatec.com.ar/")

Links encontrados: {'links': [{'type': 'Email', 'url': 'mailto:info@medytec.com'}, {'type': 'Email para Demo', 'url': 'mailto:info@medytec.com?subject=Solicitud%20de%20Demo%20Gratuita'}, {'type': 'Características', 'url': 'https://imatec.com.ar/#features'}, {'type': 'Precios', 'url': 'https://imatec.com.ar/#pricing'}, {'type': 'FAQ', 'url': 'https://imatec.com.ar/#faq'}, {'type': 'Demo', 'url': 'https://imatec.com.ar/#demo'}]}


# Folleto Informativo: Imatec

## ¿Qué es Imatec?

Imatec es una innovadora plataforma **Cloud PACS** que permite a clínicas, consultorios y centros de diagnóstico gestionar y visualizar estudios médicos de manera segura y eficiente. No requiere instalaciones complejas, lo que facilita su adopción y uso.

## ¿Cómo Funciona?

1. **📤 Subir:** Carga estudios DICOM desde cualquier dispositivo.
2. **👁️ Visualizar:** Accede al visor avanzado desde tu navegador.
3. **👥 Acceder:** Diferentes roles para médicos, pacientes y personal.
4. **🔗 Compartir:** Enlaces seguros para compartir estudios.

### Paneles por Tipo de Usuario

- **👤 Paciente:** Acceso directo a sus estudios con interfaz simplificada.
- **👨‍⚕️ Médico Informante:** Herramientas completas para diagnóstico y elaboración de informes.
- **👩‍⚕️ Médico Derivante:** Visualización de estudios solicitados y resultados.
- **💼 Secretaría/Admin:** Gestión completa de usuarios, estudios y configuración.

## Visor DICOM Avanzado

Imatec incluye un visor DICOM profesional con herramientas tales como:
- Mediciones
- Scroll y zoom
- Exportación
- Comparación y anotaciones

## Beneficios Destacados

- **📱 Accesible desde cualquier dispositivo:** Compatible con Desktop, tablet y móvil.
- **🌐 Sin instalaciones:** Funciona completamente desde el navegador.
- **👥 Roles de usuario:** Acceso personalizado por tipo de usuario.
- **🔒 Enlaces seguros:** Comparte estudios de forma segura.
- **✨ Mejor experiencia del paciente:** Interfaz intuitiva y fácil de usar.

## Planes y Precios

### Elige el plan que mejor se adapte a tu centro médico:

- **Básico - $99/mes**
  - Máquina virtual en la nube
  - 4GB RAM, 2 vCPU Intel
  - Visor DICOM
  - 100GB de espacio
  - Hasta 5 usuarios
  - Soporte por email

- **Profesional - $199/mes (Más Popular)**
  - Incluye todo en el plan básico
  - 500GB de espacio
  - Hasta 15 usuarios
  - Soporte prioritario
  - Herramientas avanzadas

- **Empresarial - Personalizado**
  - Soporte 24/7
  - Integración personalizada

## Integraciones y Soporte

Imatec se integra con sistemas de gestión hospitalaria (HIS) y otros sistemas médicos. Ofrecemos soporte técnico **24/7** vía email y chat en vivo.

## ¿Cómo Puede Ayudarte Si Eres Médico?

Si tienes una clínica, sanatorio o hospital, Imatec te ofrecerá:
- **Eficiencia en la gestión de estudios:** Ahorra tiempo en procesos administrativos.
- **Acceso rápido a información:** Facilita el diagnóstico y seguimiento de pacientes.
- **Mejor comunicación con pacientes:** Interfaz amigable que mejora la experiencia del paciente.

## Contáctanos

Para más información o para solicitar una **demo gratuita**, contáctanos a:  
- **Email:** info@medytec.com  
- **Web:** [www.medytec.com](http://www.medytec.com)  
- **Ubicación:** Buenos Aires, Argentina

---

**Imatec** - Transformando la Gestión de Estudios Médicos.  
*© 2025 Medytec. Todos los derechos reservados.*